In [3]:
import numpy as np
# import scipy.optimize as opt
# import sys, os, random, gzip
import tensorflow as tf
# from keras import backend as K
# from keras.models import *
# from keras.layers import *
# from keras.optimizers import Adam
# from keras.losses import categorical_crossentropy
# import keras
# from keras.utils import np_utils
# import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=12000)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
        print(e)

# Data 

## Custom Loss Function

In [4]:
class MeanSquaredError(tf.keras.losses.Loss):
    
    def call(self, y_true, y_pred):
        y_pred = tf.convert_to_tensor_v2(y_pred)
        y_true = tf.cast(y_true, y_pred.dtype)
        return tf.reduce_mean(math_ops.square(y_pred - y_true), axis=-1)

## Custom Layer

In [ ]:
class KQV(tf.keras.layers.Layer):
    def __init__(self, units=128):
        super(KQV, self).__init__()
        self.units = units
        
    def build(self, input_shape):  # Create the state of the layer (weights)
        wq_init = tf.random_normal_initializer()
        wk_init = tf.random_normal_initializer()
        wv_init = tf.random_normal_initializer()
        
        self.wq = tf.Variable(initial_value=wq_init(shape=(self.units, input_shape[-2] ), dtype='float32'), trainable=True)
        self.wk = tf.Variable(initial_value=wk_init(shape=(self.units, input_shape[-2] ), dtype='float32'), trainable=True)
        self.wv = tf.Variable(initial_value=wv_init(shape=(self.units, input_shape[-2] ), dtype='float32'), trainable=True)

        
#         b_init = tf.zeros_initializer()
#         self.b = tf.Variable(initial_value=b_init(shape=(self.units,), dtype='float32'), trainable=True)
        
    def call(self, inputs):  # Defines the computation from inputs to outputs
        q = tf.matmul(self.wq,inputs)
        k = tf.matmul(self.wk,inputs)
        v = tf.matmul(self.wv, inputs)

        return k, q, v


## Optimize

In [13]:
# x_(t+1) = x_t - lr*grad.(f(x_t))

opt = tf.keras.optimizers.Adam(learning_rate=0.1)
var1 = tf.Variable(10.0)
loss = lambda: (var1 ** 2)/2.0       # d(loss)/d(var1) == var1
step_count = opt.minimize(loss, [var1]).numpy()
# The first step is `-learning_rate*sign(grad)`
var1.numpy()

9.9

In [45]:
while var1.numpy()>0.1:
    opt.minimize(loss, [var1]).numpy()
    print(var1.numpy())

0.9540412
0.9305041
0.90742856
0.8848088
0.862639
0.8409134
0.8196262
0.79877156
0.7783437
0.7583367
0.73874485
0.7195623
0.70078325
0.68240196
0.66441256
0.64680934
0.6295866
0.6127385
0.59625936
0.5801435
0.5643853
0.54897904
0.5339191
0.5191999
0.50481594
0.4907616
0.47703144
0.46361995
0.4505217
0.43773136
0.4252435
0.41305286
0.40115413
0.3895421
0.3782116
0.36715743
0.35637453
0.34585783
0.33560234
0.3256031
0.3158552
0.30635378
0.29709405
0.28807122
0.2792806
0.2707176
0.26237753
0.25425592
0.24634825
0.23865008
0.23115706
0.22386485
0.21676919
0.20986587
0.20315073
0.1966197
0.19026873
0.18409383
0.1780911
0.17225665
0.16658668
0.16107745
0.15572527
0.1505265
0.14547755
0.14057492
0.13581514
0.13119482
0.12671058
0.12235916
0.11813731
0.11404186
0.11006968
0.10621771
0.102482945
0.09886242


In [53]:
a = tf.constant([[1,1],[2,2],[3,3]])
a.numpy()

array([[1, 1],
       [2, 2],
       [3, 3]], dtype=int32)

In [55]:
tf.math.pow(a,2)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1, 1],
       [4, 4],
       [9, 9]], dtype=int32)>

In [57]:
tf.reduce_sum(a, axis=-1)

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([2, 4, 6], dtype=int32)>

# 

## Symmetry Variables Function

In [98]:
## Note: m<=n
def find_N(cn):
    a = cn.shape
    a = tf.constant(a)
    n = a[-1:].numpy()
    n = n[0]
    return n

def Tile_reshape(cn):
    a = cn.shape
    a = tf.constant(a)
    b = a*0+1
    a = tf.concat([b[:-1],b[-1:]*tf.constant(a[-1])], -1)
    return a
def VP(m, cn): # m: order,  cn: input tensor, k: range
    cn = tf.cast(cn, tf.float64)
    vp = tf.math.pow(cn,m)
    vp = tf.reduce_sum(vp, axis = -1)
    vp = tf.expand_dims(vp, axis = -1)
    vp = tf.tile(vp, Tile_reshape(cn))
    return vp

# def VC(m, cn): # m: order,  cn: input tensor, k: range
#     vp = tf.math.pow(cn,m)
#     return 
def VC1(cn):
    vc = tf.reduce_sum(cn, axis = -1)
    vc = tf.expand_dims(vc, axis=-1)
    vc = tf.tile(vc, Tile_reshape(cn))
    vc = tf.cast(vc, tf.float64)
    return vc
def VC2(cn):
    vc = (VC1(cn)**2 - VP(2, cn))/2
    return vc
def VC3(cn):
    vc1 = VC1(cn)
    vp2 = VP(2,cn)
    vp3 = VP(3,cn)
    vc = (vc1**3-vp3-3*(vp2 * vc1-vp3 ))/6
    return vc
def VC4(cn):
    n = find_N(cn)
    vc = (VC3(cn)*VP(0,cn) - 3/(n-2)*VC2(cn)*VP(1,cn) + 3/(n-2)*2/(n-1)*VC1(cn)*VP(3,cn) - 3/(n-2)*2/(n-1)*VP(4,cn) )/(n-3)
    return vc


In [58]:
VC3(cn).dtype

tf.float64

In [92]:
a = cn.shape
a = tf.constant(a)
n = a[-1:].numpy()
n[0]

5

In [96]:
VC4(cn)

<tf.Tensor: shape=(3, 5), dtype=float64, numpy=
array([[ 524.,  524.,  524.,  524.,  524.],
       [   5.,    5.,    5.,    5.,    5.],
       [3554., 3554., 3554., 3554., 3554.]])>

In [97]:
cn = tf.constant([[1,2,3,4,5],[1,1,1,1,1],[3,4,5,6,7]])
# cn.numpy()
VC4(cn).numpy()

array([[ 524.,  524.,  524.,  524.,  524.],
       [   5.,    5.,    5.,    5.,    5.],
       [3554., 3554., 3554., 3554., 3554.]])

In [52]:
tf.math.multiply(VC3(cn),VC1(cn))

InvalidArgumentError: cannot compute Mul as input #1(zero-based) was expected to be a double tensor but is a int32 tensor [Op:Mul]

In [26]:
a = cn.shape
a = tf.constant(a)
b = a*0+1
a = tf.concat([b[:-1],b[-1:]*tf.constant(a[-1])], -1)
a

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 2], dtype=int32)>

In [22]:
a[-1:]

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>

In [11]:
tf.tile(cn, [1,2])

<tf.Tensor: shape=(3, 4), dtype=int32, numpy=
array([[1, 1, 1, 1],
       [2, 2, 2, 2],
       [3, 3, 3, 3]], dtype=int32)>